In [0]:
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
df = pd.read_csv('SPAM text message 20170820 - Data.csv' )

In [4]:
fig = go.Figure(go.Funnelarea(
    text = ['ham','spam'],
    values = df.groupby('Category').count().reset_index()['Message'],
    title = {"position": "top center", "text": "Message type ratio"}
    ))
fig.show()

In [0]:
df.Category.replace('ham' , 1 ,inplace = True)
df.Category.replace('spam' , 0 ,inplace = True)
df['Length'] = df.Message.str.len()

In [23]:
fig = make_subplots(
    rows=2, cols=2,
    specs=[
           [{}, {"rowspan": 2}],
           [{}, None]
    ],
    subplot_titles=("Ham","Bouth", "Spam"))

fig.add_trace(go.Histogram(x = df.query('Category == "1"')['Length'] , marker_color='#330C73'), row=1, col=1)
fig.add_trace(go.Histogram(x = df.query('Category == "1" and Length < 400')['Length'], marker_color='#636EFA'), row=1, col=2)
fig.add_trace(go.Histogram(x = df.query('Category == "0"')['Length'] , marker_color='#EF553B'), row=1, col=2)
fig.add_trace(go.Histogram(x = df.query('Category == "0"')['Length'] , marker_color='#EF553B'), row=2, col=1)

fig.update_layout(showlegend=False, title_text="Histograms of the number of characters in the message")
fig.show()

In [0]:
import re
import string
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [0]:
df['Raw_text'] = df['Message'].apply(lambda x:clean_text(x))

In [9]:
from collections import Counter
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
df['raw_split'] = df['Raw_text'].apply(lambda x:str(x).split())

def remove_stopword(x):
    return [y for y in x if y not in stopwords.words('english')]

df['raw_split'] = df['raw_split'].apply(lambda x:remove_stopword(x))

top_ham = Counter([item for sublist in df.query('Category == 1')['raw_split'] for item in sublist])
top_spam = Counter([item for sublist in df.query('Category == 0')['raw_split'] for item in sublist])
temp_ham = pd.DataFrame(top_ham.most_common(40))
temp_spam = pd.DataFrame(top_spam.most_common(40))

temp_ham.columns = ['Common_words','count']
temp_spam.columns = ['Common_words','count']

# temp.style.background_gradient(cmap='Blues')

In [0]:
temp_ham

In [0]:
!pip uninstall plotly
!pip install plotly

In [16]:
import plotly.express as px

fig = px.treemap(temp_ham, path=['Common_words'], values='count',title='Tree of Most HAM Words')
fig.show()

In [17]:
fig = px.treemap(temp_spam, path=['Common_words'], values='count',title='Tree of Most SPAM Words')
fig.show()

In [0]:
!pip install bubbly

In [28]:
from bubbly.bubbly import bubbleplot 
from plotly.offline import iplot
import chart_studio.plotly as py

figure = bubbleplot(dataset=df.head(10), x_column='Length', y_column='Length', 
    bubble_column='Length', size_column='Length', color_column='Category', 
    x_logscale=True, scale_bubble=2, height=350)

iplot(figure)


# Заменить на ЭТО!!
# import plotly.express as px
# df = px.data.gapminder()
# px.scatter(df, x="gdpPercap", y="lifeExp", animation_frame="year", animation_group="country",
#            size="pop", color="continent", hover_name="country",
#            log_x=True, size_max=55, range_x=[100,100000], range_y=[25,90])